In [15]:
#AIA_Assignemnt3
# Group: M22AI608, M22AI567, M22AI539, M22AI561, M22AI536

In [9]:
import heapq
import plotly.graph_objects as go
import numpy as np
import time
import random

# Define warehouse item names
item_names = {
    1: 'Item Type 1',
    2: 'Item Type 2',
    3: 'Item Type 3'
}

# Define time for moving between cells and recharging
TIME_PER_MOVE = 5  # seconds per move
RECHARGE_TIME = 60  # seconds (1 minute)

def a_star(grid, start, goal):
    rows, cols = len(grid), len(grid[0])
    directions = [(0,1), (1,0), (0,-1), (-1,0)]  # Right, Down, Left, Up
    
    def heuristic(a, b):
        return abs(a[0] - b[0]) + abs(a[1] - b[1])
    
    open_list = []
    heapq.heappush(open_list, (0 + heuristic(start, goal), 0, start, []))
    visited = set()
    
    while open_list:
        _, cost, current, path = heapq.heappop(open_list)
        if current in visited:
            continue
        visited.add(current)
        
        if current == goal:
            return path + [current]
        
        for direction in directions:
            next_cell = (current[0] + direction[0], current[1] + direction[1])
            if (0 <= next_cell[0] < rows and 0 <= next_cell[1] < cols and
                grid[next_cell[0]][next_cell[1]] != -1 and next_cell not in visited):
                heapq.heappush(open_list, (
                    cost + 1 + heuristic(next_cell, goal), 
                    cost + 1, 
                    next_cell, 
                    path + [current]
                ))
    return []

def robot_pathfinding(grid, charging_station, drop_off_point, battery, working_minutes):
    items = [(r, c) for r in range(len(grid)) for c in range(len(grid[0])) if grid[r][c] > 0]
    start = (0, 0)  # Starting at top-left corner
    path = []
    collected_items = []
    item_paths = {}
    visited = set()
    total_time_seconds = 0
    charge_count = 0
    item_times = {}
    recharge_times = []
    collection_time_elapsed = 0
    last_recharge_time = 0

    while items and total_time_seconds < working_minutes * 60:
        nearest_item = min(items, key=lambda item: len(a_star(grid, start, item)))
        item_path = a_star(grid, start, nearest_item)
        
        if not item_path or len(item_path) > battery:
            # If not enough battery to reach the nearest item, return to charging station
            if battery < 4:
                return_path = a_star(grid, start, charging_station)
                if not return_path:
                    break
                path.extend(return_path)
                visited.add(charging_station)
                start = charging_station
                battery = 4
                charge_count += 1
                recharge_times.append(RECHARGE_TIME)
                total_time_seconds += RECHARGE_TIME
                last_recharge_time = total_time_seconds
                continue

        # Collect item
        item_collection_time = random.randint(60, 600)  # Random time between 1 and 10 minutes
        total_time_seconds += item_collection_time
        collection_time_elapsed += item_collection_time
        item_paths[nearest_item] = item_path
        path.extend(item_path)
        path.append(nearest_item)
        collected_items.append(nearest_item)
        visited.add(nearest_item)
        start = nearest_item
        items.remove(nearest_item)
        battery -= len(item_path)
        item_times[nearest_item] = item_collection_time

        # Drop off items after collecting 2 items
        if len(collected_items) % 2 == 0:
            drop_path = a_star(grid, start, drop_off_point)
            if drop_path:
                path.extend(drop_path)
                visited.add(drop_off_point)
                start = drop_off_point  # Update starting point to drop-off point

        # Recharge if 20 minutes have passed since the last recharge
        if collection_time_elapsed - last_recharge_time >= 20 * 60:
            return_path = a_star(grid, start, charging_station)
            if return_path:
                path.extend(return_path)
                visited.add(charging_station)
                start = charging_station
                battery = 4
                charge_count += 1
                recharge_times.append(RECHARGE_TIME)
                total_time_seconds += RECHARGE_TIME
                last_recharge_time = total_time_seconds
                collection_time_elapsed = 0

    return path, collected_items, item_paths, visited, item_times, recharge_times, charge_count, total_time_seconds

def plot_grid(grid, path, collected_items, item_paths, visited, charging_station, drop_off_point):
    rows, cols = len(grid), len(grid[0])
    grid_display = np.zeros((rows, cols))

    # Fill grid_display with item values and markers for charging station, drop-off point, and obstacles
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == -1:
                grid_display[r, c] = -1  # Obstacles
            elif grid[r][c] > 0:
                grid_display[r, c] = grid[r][c]  # Items

    fig = go.Figure()

    # Plot grid
    fig.add_trace(go.Heatmap(
        z=grid_display,
        colorscale='Blues',
        zmin=-1,
        zmax=3,
        showscale=False,
        name='Grid'
    ))

    # Plot obstacles
    obs_x, obs_y = zip(*[(r, c) for r in range(rows) for c in range(cols) if grid[r][c] == -1])
    fig.add_trace(go.Scatter(
        x=obs_y,
        y=obs_x,
        mode='markers',
        marker=dict(size=15, color='black', symbol='x'),
        name='Obstacles'
    ))

    # Plot path
    path_x, path_y = zip(*path)
    fig.add_trace(go.Scatter(
        x=path_y,
        y=path_x,
        mode='markers+lines',
        marker=dict(size=8, color='red'),
        line=dict(color='red', width=2),
        name='Path'
    ))

    # Plot collected items
    if collected_items:
        items_x, items_y = zip(*collected_items)
        fig.add_trace(go.Scatter(
            x=items_y,
            y=items_x,
            mode='markers',
            marker=dict(size=12, color='green', symbol='circle'),
            name='Collected Items'
        ))

    # Plot visited cells excluding obstacles
    visited_x, visited_y = zip(*[v for v in visited if grid[v[0]][v[1]] != -1])
    fig.add_trace(go.Scatter(
        x=visited_y,
        y=visited_x,
        mode='markers',
        marker=dict(size=10, color='orange', symbol='diamond'),
        name='Visited Cells'
    ))

    # Plot charging station
    fig.add_trace(go.Scatter(
        x=[charging_station[1]],
        y=[charging_station[0]],
        mode='markers',
        marker=dict(size=15, color='blue', symbol='x'),
        name='Charging Station'
    ))

    # Plot drop-off point
    fig.add_trace(go.Scatter(
        x=[drop_off_point[1]],
        y=[drop_off_point[0]],
        mode='markers',
        marker=dict(size=15, color='red', symbol='circle'),
        name='Drop-Off Point'
    ))

    fig.update_layout(
        title='Warehouse Grid Simulation',
        xaxis=dict(title='Column Index'),
        yaxis=dict(title='Row Index', autorange='reversed'),
        width=600,
        height=600
    )
    fig.show()

def simulate_robot(grid, charging_station, drop_off_point, battery, working_minutes):
    start_time = time.time()
    path, collected_items, item_paths, visited, item_times, recharge_times, charge_count, total_time_seconds = robot_pathfinding(grid, charging_station, drop_off_point, battery, working_minutes)
    end_time = time.time()
    
    item_names_collected = [item_names[grid[r][c]] for (r, c) in collected_items]
    
    print(f"Collected Items: {item_names_collected}")
    print(f"Total Time Taken: {total_time_seconds / 60:.2f} minutes")
    print(f"Total Charging Events: {charge_count}")

    # Print item collection paths
    for item in item_paths:
        print(f"Path to collect item at {item}: {item_paths[item]}")

    # Print item collection times
    for item in collected_items:
        print(f"Time taken to collect item at {item}: {item_times.get(item, 'N/A') / 60:.2f} minutes")

    # Print recharge times
    for idx, recharge_time in enumerate(recharge_times):
        print(f"Recharge {idx + 1}: {recharge_time / 60:.2f} minutes")

    plot_grid(grid, path, collected_items, item_paths, visited, charging_station, drop_off_point)

#  grid: 0 = empty, -1 = obstacle, >0 = item , 1,2,3 are items
grid = [
    [ 0,  0,  -1,  2,  0],
    [ 0,  1,  0,  0, -1],
    [ 0,  0,  0,  2,  0],
    [ 0,  0,  3,  0,  0],
    [ 0, -1,  0,  2,  0]
]

charging_station = (4, 4)
drop_off_point = (2, 2)
battery = 4
working_minutes = 50

simulate_robot(grid, charging_station, drop_off_point, battery, working_minutes)


Collected Items: ['Item Type 1', 'Item Type 2', 'Item Type 2', 'Item Type 3', 'Item Type 2']
Total Time Taken: 26.58 minutes
Total Charging Events: 3
Path to collect item at (1, 1): [(0, 0), (0, 1), (1, 1)]
Path to collect item at (4, 3): [(4, 4), (4, 3)]
Path to collect item at (2, 3): [(2, 2), (2, 3)]
Path to collect item at (3, 2): [(4, 4), (3, 4), (3, 3), (3, 2)]
Path to collect item at (0, 3): [(4, 4), (3, 4), (2, 4), (2, 3), (1, 3), (0, 3)]
Time taken to collect item at (1, 1): 4.92 minutes
Time taken to collect item at (4, 3): 3.90 minutes
Time taken to collect item at (2, 3): 5.53 minutes
Time taken to collect item at (3, 2): 7.57 minutes
Time taken to collect item at (0, 3): 1.67 minutes
Recharge 1: 1.00 minutes
Recharge 2: 1.00 minutes
Recharge 3: 1.00 minutes


In [12]:
import pygame
import sys
import random
import heapq
from pygame.locals import *

# Initialize pygame
pygame.init()

# Screen dimensions
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
GRID_SIZE = 20  # Decreased grid size for smaller paths

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)  # Define YELLOW color

# Create screen
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Delivery Robot")

# Grid and Obstacles
def create_grid():
    grid = []
    for x in range(0, SCREEN_WIDTH, GRID_SIZE):
        for y in range(0, SCREEN_HEIGHT, GRID_SIZE):
            grid.append((x, y))
    return grid

def draw_grid():
    for x, y in grid:
        rect = pygame.Rect(x, y, GRID_SIZE, GRID_SIZE)
        pygame.draw.rect(screen, BLACK, rect, 1)

# Heuristic for A* (Manhattan distance)
def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

# A* Algorithm
def a_star_search(start, goal, obstacles):
    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}
    
    while open_set:
        current = heapq.heappop(open_set)[1]
        
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.reverse()
            return path
        
        for dx, dy in [(-GRID_SIZE, 0), (GRID_SIZE, 0), (0, -GRID_SIZE), (0, GRID_SIZE)]:
            neighbor = (current[0] + dx, current[1] + dy)
            if 0 <= neighbor[0] < SCREEN_WIDTH and 0 <= neighbor[1] < SCREEN_HEIGHT and neighbor not in obstacles:
                tentative_g_score = g_score[current] + 1
                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))
    return []

# Robot class
class Robot:
    def __init__(self):
        self.rect = pygame.Rect(GRID_SIZE, GRID_SIZE, GRID_SIZE, GRID_SIZE)
        self.color = BLUE
        self.battery = 100
        self.carrying = None
        self.path = []
        self.recharging = False  # Flag to indicate if robot is recharging

    def update(self, obstacles, charging_station):
        if self.path:
            next_move = self.path.pop(0)
            self.rect.topleft = next_move
            self.battery -= 1

        # Check battery level and task state
        if self.battery <= 30 and not self.carrying and not self.recharging:
            self.path = a_star_search(self.rect.topleft, charging_station, obstacles.values())
            self.recharging = True  # Set recharging flag

        if self.recharging and self.rect.topleft == charging_station:
            self.battery = 100  # Charge directly to 100%
            self.recharging = False  # Reset recharging flag once fully charged

        if self.battery > 0 and not self.path:
            self.recharging = False  # Reset recharging flag when not moving to recharge

    def draw(self):
        pygame.draw.rect(screen, self.color, self.rect)
        if self.carrying:
            pygame.draw.circle(screen, RED, self.rect.center, 10)
        
        # Display battery percentage or "DEAD" message
        font = pygame.font.Font(None, 24)
        if self.battery > 0:
            text_surface = font.render(f"Battery: {self.battery}%", True, BLACK)
        else:
            text_surface = font.render("DEAD", True, RED)
        screen.blit(text_surface, (10, 10))

# Define hospital-related obstacles
def generate_obstacles():
    fixed_obstacles = {
        "Obstacle_1": (6 * GRID_SIZE, 6 * GRID_SIZE),
        "Obstacle_2": (14 * GRID_SIZE, 14 * GRID_SIZE),
        "Obstacle_3": (10 * GRID_SIZE, 3 * GRID_SIZE)
    }
    
    random_obstacles = {
        f"Obstacle_{i}": (random.choice(range(0, SCREEN_WIDTH, GRID_SIZE)), random.choice(range(0, SCREEN_HEIGHT, GRID_SIZE)))
        for i in range(4, 20)  # Start from 4 to avoid overlap with fixed obstacles
    }
    
    obstacles = {**fixed_obstacles, **random_obstacles}
    return obstacles

# Environment setup
grid = create_grid()
obstacles = generate_obstacles()

# Define special locations
storage_room = (4 * GRID_SIZE, 4 * GRID_SIZE)
patient_room = (12 * GRID_SIZE, 12 * GRID_SIZE)
charging_station = (2 * GRID_SIZE, 2 * GRID_SIZE)  # Adjusted position for charging station

# Main loop
robot = Robot()
state = "DELIVERY_REQUEST"
running = True

while running:
    screen.fill(WHITE)
    draw_grid()
    
    # Draw storage room, patient room, charging station, and obstacles
    pygame.draw.rect(screen, GREEN, pygame.Rect(storage_room[0], storage_room[1], GRID_SIZE, GRID_SIZE))
    pygame.draw.rect(screen, RED, pygame.Rect(patient_room[0], patient_room[1], GRID_SIZE, GRID_SIZE))
    pygame.draw.rect(screen, BLACK, pygame.Rect(charging_station[0], charging_station[1], GRID_SIZE, GRID_SIZE))
    
    for name, obstacle_pos in obstacles.items():
        pygame.draw.rect(screen, BLACK, pygame.Rect(obstacle_pos[0], obstacle_pos[1], GRID_SIZE, GRID_SIZE))
        obstacle_font = pygame.font.Font(None, 12)  # Decreased font size for obstacle labels
        obstacle_label = obstacle_font.render(name, True, BLACK)
        screen.blit(obstacle_label, (obstacle_pos[0] + 2, obstacle_pos[1] + 2))  # Adjusted label position

    # Draw charging station
    pygame.draw.rect(screen, BLACK, pygame.Rect(charging_station[0], charging_station[1], GRID_SIZE, GRID_SIZE))
    pygame.draw.circle(screen, YELLOW, (charging_station[0] + GRID_SIZE // 2, charging_station[1] + GRID_SIZE // 2), 5)
    
    # Handle events
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            sys.exit()

    # State machine for robot actions
    if state == "DELIVERY_REQUEST":
        # Receive delivery request
        robot.path = a_star_search(robot.rect.topleft, storage_room, obstacles.values())
        state = "NAVIGATE_TO_STORAGE"
    
    elif state == "NAVIGATE_TO_STORAGE":
        if not robot.path:
            robot.carrying = "Medication"
            state = "PICK_UP_MEDICATION"
        else:
            robot.update(obstacles, charging_station)

    elif state == "PICK_UP_MEDICATION":
        robot.path = a_star_search(storage_room, patient_room, obstacles.values())
        state = "NAVIGATE_TO_PATIENT_ROOM"
    
    elif state == "NAVIGATE_TO_PATIENT_ROOM":
        if not robot.path:
            state = "DELIVER_MEDICATION"
        else:
            robot.update(obstacles, charging_station)
    
    elif state == "DELIVER_MEDICATION":
        robot.carrying = None
        robot.path = a_star_search(patient_room, charging_station, obstacles.values())
        state = "UPDATE_OBSTACLES"

    elif state == "UPDATE_OBSTACLES":
        # Example of updating obstacles
        obstacles = generate_obstacles()
        state = "DELIVERY_REQUEST"

    # Draw robot
    robot.draw()

    # Update display
    pygame.display.flip()
    pygame.time.wait(100)


SystemExit: 